In [31]:
import pandas as pd
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score,confusion_matrix, silhouette_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.datasets import make_blobs


import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


## 1. 위경도 - 지번주소 변환

In [32]:
# 픽업지 지번 주소 추출 

# apiurl = "https://api.vworld.kr/req/address"
# api_key = "57C428EF-A97E-3028-9BF6-734745493260"

# data = pd.read_csv('prj-ML-model-LT_OV30/new_data_pick_address.csv', encoding="utf-8")

# shop_results = []

# for index, row in data.iterrows():
#     lat = row['shop_loc_pnt_lat']
#     lon = row['shop_loc_pnt_lon']
    
#     # 픽업지 주소 추출
#     params_shop = {
#         "service": "address",
#         "request": "getaddress",
#         "crs": "epsg:4326",
#         "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
#         "format": "json",
#         "type": "parcel",
#         "key": api_key
#     }
    
#     response_shop = requests.get(apiurl, params=params_shop)
    
#     if response_shop.status_code == 200:
#         result_shop = response_shop.json()
#         shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
#     shop_results.append({
#         "Latitude": lat,
#         "Longitude": lon,
#         "Address": shop_address
#         })
    
# shop_result = pd.DataFrame(shop_results)
# shop_result.to_csv('prj-ML-model-LT_OV30/new_data_shop_result.csv', index=False, encoding="cp949")

In [33]:

# apiurl = "https://api.vworld.kr/req/address"
# api_key = "57C428EF-A97E-3028-9BF6-734745493260"

# data = pd.read_csv('prj-ML-model-LT_OV30/new_data_dlvry_address.csv', encoding="utf-8")

# dlvry_results = []

# for index, row in data.iterrows():
#     lat = row['dlvry_loc_pnt_lat']
#     lon = row['dlvry_loc_pnt_lon']
    
#     # 픽업지 주소 추출
#     params_shop = {
#         "service": "address",
#         "request": "getaddress",
#         "crs": "epsg:4326",
#         "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
#         "format": "json",
#         "type": "parcel",
#         "key": api_key
#     }
    
#     response_shop = requests.get(apiurl, params=params_shop)
    
#     if response_shop.status_code == 200:
#         result_shop = response_shop.json()
#         shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
#     dlvry_results.append({
#         "Latitude": lat,
#         "Longitude": lon,
#         "Address": shop_address
#         })
    
# dlvry_result = pd.DataFrame(dlvry_results)
# dlvry_result.to_csv('prj-ML-model-LT_OV30/new_data_dlvry_result.csv', index=False, encoding="cp949")

## 2. 지번주소 - 건축물대장 join

In [34]:
# dataload
#픽업지 층 데이터
pick_floor = pd.read_excel("prj-ML-model-LT_OV30/pick_floor_data.xls")

#픽업지/전달지 지번주소 데이터
pick = pd.read_csv("prj-ML-model-LT_OV30/address/new_data_shop_result.csv", encoding = "cp949")
dlvry = pd.read_csv("prj-ML-model-LT_OV30/address/new_data_dlvry_result.csv", encoding = "cp949")

# 건축물대장 데이터
info_1 = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/address/건축물정보/building_final.csv', encoding = "cp949")
info_2 = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/address/건축물대장_all.csv', encoding = "cp949")


In [35]:
info_1.rename(columns={'address':'Address'},inplace = True)
print(info_1.head(2)) # address로 pick, dlvry join

             Address  건물용도명  지상층수_max  지하층수_min  n  pick  dlvry
0  서울특별시 서초구 방배동 1-3  일반음식점       6.0         2  1     1      1
1  서울특별시 서초구 방배동 1-4   공동주택       2.0         1  1     1      1


In [36]:
info_2.rename(columns = {'대지위치' : 'Address'}, inplace = True)
print(info_2.head(2))

       건축면적  부속건축물면적  부속건축물수   건폐율  법정동코드            건물명   블록  번  외필지수  \
0  41238.98      0.0       0  4.58  10900  전투원대기숙소외 236동  NaN  1   172   
1    419.48      0.0       0  0.00  10900            NaN  NaN  1     0   

       생성일자  ...  대장종류코드명 순번  시군구코드 특수지명  착공일       연면적  총주차수  사용승인일   용적률  \
0  20220705  ...    총괄표제부  1  11650  NaN  NaN  56267.51     0    NaN  6.11   
1  20200801  ...    총괄표제부  2  11650  NaN  NaN    419.48     0    NaN  0.00   

   용적률산정연면적  
0  55056.45  
1    388.74  

[2 rows x 65 columns]


In [37]:
print(pick.shape) #1,336
print(pick.isna().sum())

(1336, 3)
Latitude     0
Longitude    0
Address      0
dtype: int64


In [38]:
print(dlvry.shape) # 17,824
print(dlvry.isna().sum()) 

(17824, 3)
Latitude     0
Longitude    0
Address      0
dtype: int64


In [39]:
info_1_pick = info_1[info_1['pick'] == 1]
info_1_dlvry = info_1[info_1['dlvry'] == 1]

print(info_1_pick.columns)
print(pick.columns)

Index(['Address', '건물용도명', '지상층수_max', '지하층수_min', 'n', 'pick', 'dlvry'], dtype='object')
Index(['Latitude', 'Longitude', 'Address'], dtype='object')


## 2-1. info_1 join

In [40]:
pick = pd.merge(pick, info_1_pick[['Address', '건물용도명']], 
                left_on = ['Address'], right_on = ['Address'], how = 'left')

print(pick.isna().sum()) #183개
print(pick.shape)

Latitude       0
Longitude      0
Address        0
건물용도명        183
dtype: int64
(1336, 4)


In [41]:
print(info_1_dlvry.columns)

Index(['Address', '건물용도명', '지상층수_max', '지하층수_min', 'n', 'pick', 'dlvry'], dtype='object')


In [42]:
dlvry = pd.merge(dlvry, info_1_dlvry[['Address', '건물용도명', '지상층수_max', '지하층수_min']], 
                left_on = ['Address'], right_on = ['Address'], how = 'left')

print(dlvry.isna().sum()) # 5,125개
print(dlvry.shape)

Latitude        0
Longitude       0
Address         0
건물용도명        5125
지상층수_max     5125
지하층수_min     5125
dtype: int64
(17824, 6)


## 2-2. info_2 join

In [43]:
info_2 = info_2[["주용도코드명", "Address"]]
pick = pd.merge(pick, info_2, 
                left_on="Address", right_on="Address", how="left")

dlvry = pd.merge(dlvry, info_2, 
                 left_on="Address", right_on="Address", how="left")


pick["건물용도명_2"] = pick.apply(lambda row: row["주용도코드명"] if pd.isna(row["건물용도명"]) else row["건물용도명"], axis=1)
dlvry["건물용도명_2"] = dlvry.apply(lambda row: row["주용도코드명"] if pd.isna(row["건물용도명"]) else row["건물용도명"], axis=1)


In [44]:
print(pick.isna().sum()) # 169
print(pick.shape) #1336

Latitude        0
Longitude       0
Address         0
건물용도명         183
주용도코드명       1292
건물용도명_2       169
dtype: int64
(1336, 6)


In [45]:
print(dlvry.isna().sum()) # 3823
print(dlvry.shape) #17835

Latitude         0
Longitude        0
Address          0
건물용도명         5131
지상층수_max      5131
지하층수_min      5131
주용도코드명       13793
건물용도명_2       3823
dtype: int64
(17835, 8)


## 2-3. 픽업지 층수 데이터 join

In [46]:
pick_floor.rename(columns = {'longitude' : 'Longitude', 'latitude' : 'Latitude', 'addr' : 'Address'}, inplace = True)
print(pick_floor.shape) #2935

(2935, 10)


In [47]:
print(pick_floor.columns)
print(pick.columns)

Index(['shop_no', 'Address', 'floor', 'rgn1_nm', 'rgn2_nm', 'rgn3_nm',
       'shop_category', 'Latitude', 'Longitude', 'PICK'],
      dtype='object')
Index(['Latitude', 'Longitude', 'Address', '건물용도명', '주용도코드명', '건물용도명_2'], dtype='object')


In [48]:
# 픽업지 건물정보 join
pick_floor = pd.merge(pick_floor, pick[['Latitude', 'Longitude', '건물용도명_2']],
                      left_on=['Latitude', 'Longitude'], right_on=['Latitude', 'Longitude'], how='left')
print(pick_floor.isna().sum())
print(pick_floor.shape) #2,935

shop_no             0
Address             0
floor               0
rgn1_nm             0
rgn2_nm             0
rgn3_nm             0
shop_category       0
Latitude            0
Longitude           0
PICK                0
건물용도명_2          1272
dtype: int64
(2935, 11)


## 3. raw_data join

In [49]:
data = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/new_data.csv', encoding = 'utf-8')
print(data.head(2))
print(data.shape) #27654

            dlvry_id business_day  reg_hour service_type   shop_no  ord_price  \
0  20230916_1540YBKT   2023-09-16        22          배민1  14150430      17500   
1  20230915_6952RRHU   2023-09-15        11          배민1  10643316      62500   

   notiOver  actual_dlvry_distance  shop_loc_pnt_lat  shop_loc_pnt_lon  ...  \
0         0                    566         37.481687        126.992855  ...   
1         0                    551         37.505045        127.022297  ...   

   dlvry_loc_pnt_lon  추천노출시간_AI  추천노출시간_일반   추천미발생시간유  추천미발생시간무  배차후취소시간  \
0         126.992994   0.416667        0.0  21.950000  2.383333      0.0   
1         127.024574   0.850000        0.0   0.333333  0.166667      0.0   

   전체배차시간  notiOver_min_max  per_display  per_no_recomm  
0   24.75               0.0     1.683502      98.316498  
1    1.35               0.0    62.962963      37.037037  

[2 rows x 21 columns]
(27654, 21)


In [50]:
# raw_data + 픽업지 정보
data = pd.merge(data, pick_floor[['shop_no', 'floor', 'rgn2_nm', 'rgn3_nm', 'shop_category', '건물용도명_2', 'Address']], on='shop_no', how='left')
data = data.rename(columns={
    'floor': 'pick_floor',
    'rgn2_nm': 'pick_rgn2_nm',
    'rgn3_nm': 'pick_rgn3_nm',
    '건물용도명_2': 'pick_건물용도',
    'shop_category': 'pick_category',
    'Address': 'pick_address'
})

In [51]:
dlvry['Latitude'] = pd.to_numeric(dlvry['Latitude'], errors='coerce')
dlvry['Longitude'] = pd.to_numeric(dlvry['Longitude'], errors='coerce')
data['dlvry_loc_pnt_lat'] = pd.to_numeric(data['dlvry_loc_pnt_lat'], errors='coerce')
data['dlvry_loc_pnt_lon'] = pd.to_numeric(data['dlvry_loc_pnt_lon'], errors='coerce')

dlvry['Latitude'] = dlvry['Latitude'].round(7).astype(str)
dlvry['Longitude'] = dlvry['Longitude'].round(7).astype(str)
data['dlvry_loc_pnt_lat'] = data['dlvry_loc_pnt_lat'].round(7).astype(str)
data['dlvry_loc_pnt_lon'] = data['dlvry_loc_pnt_lon'].round(7).astype(str)

In [52]:
# raw_data + 전달지 정보
data = pd.merge(data, dlvry[['Latitude', 'Longitude', 'Address', '지상층수_max', '지하층수_min', '건물용도명_2']],
                left_on=['dlvry_loc_pnt_lat', 'dlvry_loc_pnt_lon'], right_on=['Latitude', 'Longitude'], how='left')
data = data.rename(columns={
    'Address': 'dlvry_address',
    '지상층수_max': 'dlvry_지상층수',
    '지하층수_min': 'dlvry_지하층수',
    '건물용도명_2': 'dlvry_건물용도'
})

In [53]:
print(data.isna().sum())

dlvry_id                    0
business_day                0
reg_hour                    0
service_type                0
shop_no                     0
ord_price                   0
notiOver                    0
actual_dlvry_distance       0
shop_loc_pnt_lat            0
shop_loc_pnt_lon            0
dlvry_loc_pnt_lat           0
dlvry_loc_pnt_lon           0
추천노출시간_AI                   0
추천노출시간_일반                   0
추천미발생시간유                    0
추천미발생시간무                    0
배차후취소시간                     0
전체배차시간                      0
notiOver_min_max            0
per_display                 0
per_no_recomm               0
pick_floor                115
pick_rgn2_nm              115
pick_rgn3_nm              115
pick_category             115
pick_건물용도                3235
pick_address              115
Latitude                 1214
Longitude                1214
dlvry_address            1214
dlvry_지상층수               9736
dlvry_지하층수               9736
dlvry_건물용도               7135
dtype: int

In [54]:
cols_to_replace_na = ["pick_floor", "pick_rgn2_nm", "pick_rgn3_nm", "pick_category", "pick_건물용도", "pick_address", "dlvry_address", "dlvry_지상층수", "dlvry_지하층수", "dlvry_건물용도"]
for col in cols_to_replace_na:
    data[col] = data[col].where(data[col] != 'NA')

In [55]:
data = data[(data['pick_건물용도'].notna()) & (data['dlvry_건물용도'].notna())]
print(data.shape) # 18213
print(data.isna().sum()) 


(18213, 33)
dlvry_id                    0
business_day                0
reg_hour                    0
service_type                0
shop_no                     0
ord_price                   0
notiOver                    0
actual_dlvry_distance       0
shop_loc_pnt_lat            0
shop_loc_pnt_lon            0
dlvry_loc_pnt_lat           0
dlvry_loc_pnt_lon           0
추천노출시간_AI                   0
추천노출시간_일반                   0
추천미발생시간유                    0
추천미발생시간무                    0
배차후취소시간                     0
전체배차시간                      0
notiOver_min_max            0
per_display                 0
per_no_recomm               0
pick_floor                  0
pick_rgn2_nm                0
pick_rgn3_nm                0
pick_category               0
pick_건물용도                   0
pick_address                0
Latitude                    0
Longitude                   0
dlvry_address               0
dlvry_지상층수               2310
dlvry_지하층수               2310
dlvry_건물용도                  

## 4. is_holiday, 요일 파생변수 생성

In [56]:
holiday_list = pd.to_datetime([
    "2022-01-01", "2022-01-31", "2022-02-01", "2022-03-01", "2022-03-09",
    "2022-05-05", "2022-05-08", "2022-06-01", "2022-06-06", "2022-08-15",
    "2022-09-09", "2022-09-10", "2022-09-11", "2022-09-12", "2022-10-03",
    "2022-10-09", "2022-10-10", "2022-12-25", "2023-01-01", "2023-01-21",
    "2023-01-22", "2023-01-23", "2023-01-24", "2023-03-01", "2023-05-01",
    "2023-05-05", "2023-05-27", "2023-05-29", "2023-06-06", "2023-08-15",
    "2023-09-28", "2023-09-29", "2023-09-30", "2023-10-03", "2023-10-09",
    "2023-12-25"
])

# 요일
data['business_day'] = pd.to_datetime(data['business_day'])
data.loc[:, 'day_of_week'] = data['business_day'].dt.day_name().str[:3]
data.loc[:, 'is_holiday'] = data['business_day'].isin(holiday_list) | data['day_of_week'].isin(["Fri", "Sat", "Sun"])

print(data['day_of_week'].value_counts())
print(data['is_holiday'].value_counts())

day_of_week
Sat    6338
Sun    6120
Fri    5755
Name: count, dtype: int64
is_holiday
True    18213
Name: count, dtype: int64


## 5. weather data join

In [57]:
weather_new = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/new_data_weather.csv', encoding = 'cp949')
weather_new['일시'] = pd.to_datetime(weather_new['일시'])
weather_new['시간'] = weather_new['일시'].dt.hour
weather_new['일시'] = weather_new['일시'].dt.strftime('%Y-%m-%d')
weather = weather_new.fillna(0)

weather = weather[(weather['시간'] >= 8) & (weather['시간'] <= 22)]
weather.rename(columns={"기온(°C)":'기온',"강수량(mm)":'강수량'},inplace=True)
weather = weather[['일시', '기온', '강수량', '시간']]
print(weather.head())

            일시    기온  강수량  시간
8   2023-09-14  20.1  0.0   8
9   2023-09-14  22.7  0.0   9
10  2023-09-14  24.3  0.0  10
11  2023-09-14  25.9  0.0  11
12  2023-09-14  27.0  0.0  12


In [58]:
data['business_day'] = data['business_day'].dt.strftime('%Y-%m-%d')
merged_data = data.merge(weather, left_on='business_day', right_on='일시')
filtered_data = merged_data[(merged_data['reg_hour']-1 == merged_data['시간'])]

print(filtered_data.shape) # 18213 

(18213, 39)


In [59]:
print(filtered_data.columns)

Index(['dlvry_id', 'business_day', 'reg_hour', 'service_type', 'shop_no',
       'ord_price', 'notiOver', 'actual_dlvry_distance', 'shop_loc_pnt_lat',
       'shop_loc_pnt_lon', 'dlvry_loc_pnt_lat', 'dlvry_loc_pnt_lon',
       '추천노출시간_AI', '추천노출시간_일반', '추천미발생시간유', '추천미발생시간무', '배차후취소시간', '전체배차시간',
       'notiOver_min_max', 'per_display', 'per_no_recomm', 'pick_floor',
       'pick_rgn2_nm', 'pick_rgn3_nm', 'pick_category', 'pick_건물용도',
       'pick_address', 'Latitude', 'Longitude', 'dlvry_address', 'dlvry_지상층수',
       'dlvry_지하층수', 'dlvry_건물용도', 'day_of_week', 'is_holiday', '일시', '기온',
       '강수량', '시간'],
      dtype='object')


In [60]:
filtered_data.to_csv("prj-ML-model-LT_OV30/new_data_final.csv", encoding="utf-8", index=False)